In [16]:
print(os.getcwd())

D:\pyhon-workspace\1인 가구


In [18]:
import aihub_embedding
print(dir(aihub_embedding))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [5]:
# Wav2Vec2 모델 로드 및 임베딩 함수 정의
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
import torchaudio

# 모델 불러오기
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 오디오 로딩 함수
def load_audio(file_path):
    waveform, sr = torchaudio.load(file_path)
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)
    return waveform.squeeze()

# 임베딩 추출 함수
def extract_embedding(wav_path):
    try:
        waveform = load_audio(wav_path)
        inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = model(**inputs.to(device))
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        return embedding
    except Exception as e:
        print(f"❌ 오류: {wav_path}, {e}")
        return None

C:\Users\asia\.conda\envs\py312tf9\Lib\site-packages\transformers\configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [8]:
import os

# 예시: 단일 예측을 위해 사용할 오디오 경로를 실제 파일로 지정
merged_path = os.path.join("감정 분류를 위한 대화 음성 데이터셋", "4차년도", "your_audio_file.wav")

# 실제 존재하는지 확인
print(os.path.exists(merged_path))  # True이면 OK

False
['.ipynb_checkpoints', '15초 단위 audio chunk 저장하기.ipynb', 'aihub 병합 및 맵핑.ipynb', 'aihub 임베딩.ipynb', 'aihub 학습 및 평가 .ipynb', 'aihub_embedding.py', 'aihub_emotion_preprocessed_cleaned.csv', 'brighter-datasetBRIGHTER-emotion-categories.ipynb', 'diarized_chunks', 'EmoGator-main', 'EmoGatormp32wav.ipynb', 'EmoGator_metadata.csv', 'emotion_classifier.pkl', 'emotion_classifier.pth', 'emotion_faiss.index', 'emotion_label_encoder.pkl', 'emotion_X.npy', 'emotion_X_no_disgust.npy', 'emotion_y.npy', 'emotion_y_no_disgust.npy', 'faiss 인덱스 활용 방식 요약.ipynb', 'faiss.ipynb', 'faiss_metadata.pkl', 'label_encoder_classes.npy', 'madecsv.ipynb', 'makecsv.py', 'temp_chunks', 'user_reference', 'user_segments_merged.wav', 'wav mapping check.ipynb', 'wav2vec2_embeddings.pkl', '__pycache__', '감정 분류를 위한 대화 음성 데이터셋', '감정예측 및 로그 저장.ipynb', '멀티라벨 벡터화 + 데이터셋 준비.ipynb', '신호 전처리.ipynb', '임베딩 코드.ipynb', '자료', '화자 분류 코드.ipynb']


In [13]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from datetime import datetime
import sys

# 1. MLP 모델 구조 정의 (학습 당시와 동일해야 함)
class EmotionMLP(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256, num_classes=4):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.model(x)

# 2. 감정 라벨 (학습 시 사용한 순서와 동일하게)
emotion_labels = ['anger', 'happiness', 'neutral', 'sadness']

# 3. 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 4. 모델 불러오기
emotion_model = EmotionMLP().to(device)
emotion_model.load_state_dict(torch.load("emotion_classifier.pth", map_location=device))
emotion_model.eval()

# 5. 음성 임베딩 함수 (기존 wav2vec2 기반 함수 호출 필요)
sys.path.append(".")
from aihub_embedding import extract_embedding  # 이미 작성된 임베딩 함수

# 예측 대상 오디오 경로
merged_path = "./user_segments_merged.wav"

# 6. 감정 예측
with torch.no_grad():
    embedding = extract_embedding(merged_path)  # shape: (768,)
    tensor_input = torch.tensor(embedding).unsqueeze(0).to(device).float()
    logits = emotion_model(tensor_input)
    pred = torch.argmax(logits, dim=1).item()
    predicted_emotion = emotion_labels[pred]

# 7. 로그 저장
log_df = pd.DataFrame([{
    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "file": os.path.basename(merged_path),
    "predicted_emotion": predicted_emotion
}])
log_df.to_csv("emotion_log.csv", index=False)

print(f"✅ 예측된 감정: {predicted_emotion}")
print("📄 emotion_log.csv에 저장 완료")

ImportError: cannot import name 'extract_embedding' from 'aihub_embedding' (D:\pyhon-workspace\soundofmind\aihub_embedding.py)